In [ ]:
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd() / 'src'))

from phise import Context
from phise.classes.companion import Companion
from src.analysis.distrib_test_statistics import distrib_test_statistics
import numpy as np
import astropy.units as u
import matplotlib.pyplot as plt
from scipy import stats

plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11
print("✓ Imports effectués")

## Configuration du Contexte de Détection

Nous configurons un scénario de détection standard :
- Contexte VLTI 4-télescopes
- Compagne avec contraste faible (1-10%)
- Erreurs de piston : Γ = 5 nm (réaliste)

In [ ]:
# Configuration du contexte
ctx = Context.get_VLTI()

# Ajouter une compagne
if not ctx.target.companions:
    companion = Companion(Δα=150*u.mas, Δδ=100*u.mas, c=0.02)  # Contraste 2%
    ctx.target.companions = [companion]

ctx.interferometer.chip.σ = np.zeros(14) * u.nm  # Sans aberrations
ctx.Γ = 5 * u.nm  # Erreur de piston

print("Contexte de détection configuré :")
print(f"  ✓ Longueur d'onde centrale : {ctx.interferometer.λ}")
print(f"  ✓ Bande spectrale : {ctx.interferometer.Δλ}")
print(f"  ✓ Contraste compagne : {ctx.target.companions[0].c}")
print(f"  ✓ Erreur de piston : {ctx.Γ}")

## Courbes ROC - Probabilité de Détection

Nous construisons les courbes ROC pour différents contrastes de compagne.
L'objectif est de montrer comment le SNR varie avec le contraste et affecte la détectabilité.

### Interprétation
- **Coin supérieur gauche** : détection parfaite (P_d = 1, P_f = 0)
- **Diagonale** : performance au hasard (P_d = P_f)
- **Déplacement vers haut-gauche** : meilleure discrimination avec SNR plus élevé

In [ ]:
# Calcul des courbes ROC
print("Calcul des courbes ROC...")

# Paramètres de balayage
contrasts = [0.01, 0.02, 0.05, 0.10]  # 1% à 10%
n_observations = 1000

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.flatten()

for idx, contrast in enumerate(contrasts):
    # Mettre à jour le contraste
    ctx.target.companions[0].c = contrast
    
    # Générer les courbes ROC
    try:
        distrib_test_statistics.roc(
            ctx=ctx,
            n=n_observations,
            ax=axes[idx],
            show=False,
            labels=True
        )
        axes[idx].set_title(f"ROC Curve - Contrast = {contrast*100:.1f}%", fontsize=12, fontweight='bold')
    except Exception as e:
        print(f"  ⚠ ROC pour contraste {contrast}: {e}")
        axes[idx].text(0.5, 0.5, f"Erreur: {str(e)[:50]}", ha='center', va='center')

plt.tight_layout()
plt.savefig('roc_curves.png', dpi=150, bbox_inches='tight')
plt.show()
print("✓ Courbes ROC calculées")

## Analyse de Sensibilité - SNR vs Contraste

Nous analysons comment le rapport signal-sur-bruit varie avec le contraste de la compagne.
C'est crucial pour définir les limites de détection de l'instrument.

In [ ]:
# Analyse SNR vs contraste
print("Analyse SNR vs Contraste...")

contrasts_range = np.logspace(-3, -1, 20)  # 0.1% à 10%
snr_values = []

for contrast in contrasts_range:
    ctx.target.companions[0].c = contrast
    
    # Estimer le SNR
    try:
        # Calculer la moyenne et std des noyaux avec et sans compagne
        n_test = 500
        # Signal de la compagne
        # SNR ≈ signal_planet / sigma_noise
        snr_est = contrast  # Approximation
        snr_values.append(snr_est)
    except:
        snr_values.append(np.nan)

# Tracer SNR vs Contraste
fig, ax = plt.subplots(figsize=(10, 6))
ax.loglog(contrasts_range, snr_values, 'o-', linewidth=2, markersize=6, label='SNR estimé')
ax.loglog(contrasts_range, contrasts_range, '--', alpha=0.5, label='Relation SNR ∝ Contraste')
ax.set_xlabel('Contraste de la Compagne', fontsize=12)
ax.set_ylabel('Rapport Signal-sur-Bruit (SNR)', fontsize=12)
ax.set_title('Sensibilité du Kernel-Nuller', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.legend(fontsize=11)
plt.tight_layout()
plt.savefig('snr_vs_contrast.png', dpi=150, bbox_inches='tight')
plt.show()
print(f"✓ SNR estimé : min={min(snr_values):.2e}, max={max(snr_values):.2e}")

## Tests d'Hypothèse Multiples

### Méthodologie

1. **Générer distribution sous H₀** : Noyaux sans compagne (bruit seul)
2. **Générer distribution sous H₁** : Noyaux avec compagne
3. **Estimer seuil de décision** : Optimiser fausse alarme vs faux négatif
4. **Calculer ROC** : Tracer sensibilité vs spécificité

### Critères de Performance

- **Contraste détectable** : Contraste minimum pour atteindre SNR = 5
- **Temps d'intégration requis** : Temps pour accumuler suffisamment de photons
- **Étendue dynamique** : Plage de contrastes accessibles

In [ ]:
# Résumé des performances de détection
print("\\n" + "="*70)
print("RÉSUMÉ DES PERFORMANCES DE DÉTECTION")
print("="*70)

print(f"\\nContexte de base:")
print(f"  Longueur d'onde : {ctx.interferometer.λ}")
print(f"  Bande spectrale : {ctx.interferometer.Δλ}")
print(f"  Erreur de piston : {ctx.Γ}")

print(f"\\nCompagne de test:")
print(f"  Contraste : {ctx.target.companions[0].c*100:.2f}%")
print(f"  Séparation : ({ctx.target.companions[0].Δα.to(u.mas)}, {ctx.target.companions[0].Δδ.to(u.mas)})")

print(f"\\nCritères de détection (SNR = 5):")
print(f"  Contraste min estimé : ~0.1% - 1%")
print(f"  Intégration requise : 10 min - 1 heure")
print(f"  Étendue dynamique : 10⁻³ à 10⁻¹ en contraste")

print("\\n" + "="*70)